In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [3]:
# Parameters
FILE_PATH = './Dataset/EURUSD/EURUSD_M30_features+label_v.2.1.csv'
seq_length = 15
num_layers = 2
num_heads = 4
batch_size = 1024
epochs = 50
dropout = 0.4
learning_rate = 0.005

In [4]:
# Load and preprocess the data
data = pd.read_csv(FILE_PATH)
columns = ['Close', 'SMA200', 'SMA50', 'RSI14']
label_column = ['signal']
scaler = MinMaxScaler()

In [5]:
data[columns] = scaler.fit_transform(data[columns])
features = data[columns].values
labels = data[label_column].values

In [6]:
# Create PyTorch Dataset
class ForexDataset(Dataset):
    def __init__(self, features, labels, seq_length):
        self.features = features
        self.labels = labels
        self.seq_length = seq_length

    def __len__(self):
        return len(self.features) - self.seq_length

    def __getitem__(self, idx):
        x = self.features[idx:idx + self.seq_length]
        y = self.labels[idx + self.seq_length]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

In [7]:
dataset = ForexDataset(features, labels, seq_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [8]:
# Define Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, input_dim, seq_length, num_heads, num_layers, dropout):
        super(TransformerModel, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(seq_length * input_dim, 3)  # Adjusted to match the flattened size

    def forward(self, x):
        x = self.transformer_encoder(x)
        x = x.permute(1, 0, 2)  # Reshape to (batch_size, seq_length, input_dim)
        x = x.reshape(x.size(0), -1)  # Flatten to (batch_size, seq_length * input_dim)
        x = self.fc(x)
        return x

In [ ]:
input_dim = len(columns)
model = TransformerModel(input_dim, seq_length, num_heads, num_layers, dropout)

In [ ]:
# Training setup
device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    
    
    for batch_features, batch_labels in dataloader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        batch_features = batch_features.permute(1, 0, 2)  # Shape: (seq_length, batch_size, input_dim)
        batch_labels = batch_labels.squeeze()  # Shape: (batch_size)
        assert batch_labels.min() >= 0 and batch_labels.max() <= 2, "Labels contain invalid values."

        optimizer.zero_grad()
        outputs = model(batch_features)  # Shape: (batch_size, num_classes)
        print("Outputs shape:", outputs.shape)
        print("Batch labels shape:", batch_labels.shape)

        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader):.4f}")
    
    
    
    

